In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"
import keras
import keras.ops as K
from keras.layers import Input, Flatten, Dense
from keras.optimizers import Adam
from keras.metrics import BinaryAccuracy

# from keras.models import Sequential
from deel.lip.model import Sequential

from deel.lip.layers import (
    SpectralDense,
    SpectralConv2D,
    ScaledL2NormPooling2D,
    FrobeniusDense,
)
from deel.lip.activations import GroupSort, GroupSort2
from deel.lip.losses import HKR, KR, HingeMargin, MulticlassHKR, MulticlassKR

import numpy as np
import decomon

from data_processing import load_data, select_data_for_radius_evaluation_MNIST08
from radius_evaluation_tools import compute_binary_certificate, starting_point_dichotomy
from lipschitz_decomon_tools import get_local_maximum, echantillonner_boule_l2_simple

# Data Loading

In [2]:
x_train, x_test, y_train, y_test, y_test_ord = load_data("MNIST08")

In [3]:
model_path = "/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output.keras"
model = keras.models.load_model(model_path)
model.compile(
   
    loss=HKR(
        alpha=10.0, min_margin=1.0
    ),  # HKR stands for the hinge regularized KR loss
    metrics=[
        # KR,  # shows the KR term of the loss
        HingeMargin(min_margin=1.0),  # shows the hinge term of the loss
    ],
    optimizer=Adam(learning_rate=0.001),)

model_bis = keras.models.load_model("/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output_converted_4logits.keras")
model_bis.compile(
        # decreasing alpha and increasing min_margin improve robustness (at the cost of accuracy)
        # note also in the case of lipschitz networks, more robustness require more parameters.
        loss=MulticlassHKR(alpha=100, min_margin=0.25),
        optimizer=Adam(1e-4),
        metrics=["accuracy", MulticlassKR()],)

images, labels, idx_list = select_data_for_radius_evaluation_MNIST08(x_test, y_test_ord, model_bis)

/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 12 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


# Selection of studied data

In [ ]:
pt_choosen = 0
x = images[pt_choosen:pt_choosen+1].flatten().detach().cpu().numpy()
label = labels[pt_choosen:pt_choosen+1]
eps=0.5

In [5]:
yi = echantillonner_boule_l2_simple(x,eps)

# Dataset Creation

We want to learn an affine model that is close and overapproximate norm(x,y), hence, we create a dataset with generated samples

In [6]:
def norm(x,y):
    return np.linalg.norm(x-y)

In [7]:
def create_dataset(nb):    
    input = []
    label = []
    for _ in range(nb):
        x_current = echantillonner_boule_l2_simple(x,eps)
        input.append(x_current)
        label.append(norm(x_current, yi))
    return np.array(input), np.array(label)

In [8]:
inputs, labels = create_dataset(2048)

# Model Training
We create a single layer affine model without activations and train it with a custom loss

In [52]:
BATCH_SIZE = 128
EPOCHS = 300  # Nombre d'époques (ajustable)
STEPS_PER_EPOCH = 50 # Nombre de batches par époque (car le générateur est infini)
LEARNING_RATE = 0.01
LAMBDA_PENALTY_TF = 100 # Coefficient de pénalité

In [53]:
# --- 1. Définition du Modèle Affine (Keras) ---
def create_affine_model(input_dim_model):
    model = keras.Sequential([
        keras.layers.Dense(1, activation=None, input_shape=(input_dim_model,), name="affine_layer")
    ], name="simple_affine_network")
    return model

In [54]:
def sur_approximation_mse_loss(y_true_norm, y_pred_affine):
    # y_pred_affine: Sortie du modèle (Wx + b_nn), shape [batch_size, 1]
    # y_true_norm: Norme L2 cible (||x - x0||), shape [batch_size,]
    
    y_pred_affine_squeezed = K.squeeze(y_pred_affine) # Shape [batch_size,]
    
    # Terme 1 (MSE): (g_i - f_i)^2, où g_i = y_pred_affine, f_i = y_true_norm
    mse_gap_term = K.square(y_pred_affine_squeezed - y_true_norm)
    
    # Terme 2 (Pénalité): lambda * ReLU(f_i - g_i)^2
    violation = y_true_norm - y_pred_affine_squeezed # Positif si g_i < f_i (violation)
    penalty_term = LAMBDA_PENALTY_TF * K.square(K.relu(violation))
    
    # Perte moyenne sur le batch
    loss = K.mean(mse_gap_term + penalty_term)
    return loss

In [55]:
x_train[0].flatten().shape[0]

784

In [56]:
model = create_affine_model(x_train[0].flatten().shape[0])
model.compile(optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=sur_approximation_mse_loss)

In [57]:
model.fit(inputs, labels,
                    epochs=EPOCHS,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    verbose=1) # verbose=1 pour la barre de progression

Epoch 1/300
34/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.9349

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.3325
Epoch 2/300
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2374
Epoch 3/300
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1072
Epoch 4/300
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0822
Epoch 5/300
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0788
Epoch 6/300
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0674
Epoch 7/300
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0707
Epoch 8/300
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0702
Epoch 9/300
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0875
Epoch 10/300
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0700
Epoch 11/300
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0723
Epoch 12/300
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1460
Epoch 13/300
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.4077
Epoch 14/300
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0827
Epoch 15/300
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0744
Epoch 16/300
50/

is the hyperplane over the norm ?

In [58]:
i = 4
model(inputs[i][None]) - norm(inputs[i], yi)

tensor([[0.1704]], device='cuda:0', grad_fn=<SubBackward0>)

In [59]:
W, b = model.get_weights()

In [79]:
np.max(W)

0.18306985

In [80]:
b

array([0.00291549], dtype=float32)

In [62]:
W[:,0]@(inputs[i])+b

array([0.47401604], dtype=float32)

In [63]:
model(inputs[i][None])

tensor([[0.4740]], device='cuda:0', grad_fn=<AddBackward0>)

# Empirical Tests

In [179]:
x_current = echantillonner_boule_l2_simple(x,eps)

if (norm(x_current, yi) - model(x_current[None]))<0:
    print('the hyperplan is overapproximating : ', norm(x_current, yi) - model(x_current[None]))
else:
    print("error")



the hyperplan is overapproximating :  tensor([[-0.0714]], device='cuda:0', grad_fn=<RsubBackward1>)


# Optimisation

We know want to solve the optimisation problem, in order to know if our hyperplane is valid

In [71]:
from scipy.optimize import minimize

In [72]:
def function_to_optimize(x, x0, yi, W, b, eps):
    z0 = x0-yi
    return np.linalg.norm(z0) + eps + 2*z0@x - W@(z0 + x + yi) - b

In [73]:
# Define the constraint: ||x - x_centre||_2**2 <= eps**2
def unit_ball_constraint(x, x_ball_center, eps):
    return eps - np.linalg.norm(x - x_ball_center)

def jacobian_unit_ball_constraint(x, x_ball_center, eps):
    return -2*(x - x_ball_center)

In [74]:
x_ball_center = x
x_ball_center = np.asarray(x_ball_center, dtype=np.float64)

args_contrainte = (x_ball_center, eps)
# Set up the constraint dictionary
constraints = ({
    'type': 'eq',  # Inequality constraint: constraint(x) >= 0
    'fun': unit_ball_constraint,
    # 'jac': jacobian_unit_ball_constraint,
    'args': args_contrainte
})

In [75]:
result = minimize(fun=lambda x :-function_to_optimize(x, x_ball_center, yi, W.squeeze(1), b, eps),\
        # jac= lambda x :-jac_function_to_optimize(x, label, W_list, b_list, y_list, model, L),\
        x0 = x_ball_center, method='SLSQP', constraints=constraints)

In [76]:
if result.success:
    print(result.x, -result.fun)
else:
    print("Optimization failed:", result.message)   

[0.01785714 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714
 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714
 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714
 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714
 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714
 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714
 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714
 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714
 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714
 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714
 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714
 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714
 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714
 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714
 0.01785714 0.01785714 0.01785714 0.01785714 0.01785714 0.0178

In [77]:
-result.fun

-0.4208584129810333

In [ ]:
def f(z, W, b, y, label, L=1):
    if label==0:
        return model(y.reshape((1,28,28))[None]).cpu().detach().numpy()[0,0] +\
                L*(W@z +b) #scalar
    else:
        return model(y.reshape((1,28,28))[None]).cpu().detach().numpy()[0,0] -\
                L*(W@z +b) #scalar


In [ ]:
def f_all(z, W_list, b_list, y_list, label, L=1):
    output = []
    for i in range(len(y_list)):
        output.append(f(z,W_list[i], b_list[i], y_list[i], label, L))
    if label==0:
        return np.max(output)
    else:
        return np.min(output)

Can we retrieve this gap to b in order to get a better hyperplane ?